In [ ]:
%pip install azure-ai-contentsafety
%pip install python-dotenv

In [112]:
from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [114]:
apikey = os.getenv("AZURE_API_KEY")
endpoint = os.getenv("AZURE_ENDPOINT")

In [115]:
credential = AzureKeyCredential(apikey)
client = ContentSafetyClient(endpoint, credential)

In [116]:
API_KEY = os.getenv("PORTAL_API_KEY")
SECRET_KEY = os.getenv("PORTAL_SECRET_KEY")

In [117]:
import requests

result = requests.post(
    os.getenv("PORTAL_TOKEN_AUTH"),
    json={"data": {"key": API_KEY, "secret": SECRET_KEY}},
)

In [ ]:
print(f"Status code: {result.status_code}")
print(f"Content: {result.json()}")

data = result.json()
token = data['data']['token']

token

In [119]:
endpoint = os.getenv("PORTAL_DATA_ENDPOINT")

In [120]:
headers = {"Authorization": f"Bearer {token}"}

data = requests.get(endpoint, headers=headers).json()

data = data["data"]

In [121]:
texts = []
images_urls = []

for post in data:
    if post["content"][0] != "#":
        texts.append(post["content"])

    media = post["media"]
    if media is not None:
        images_urls.append(media["photo"]["url"])

In [ ]:
print(texts)
print(images_urls)

In [123]:
from azure.ai.contentsafety.models import TextCategory
from azure.ai.contentsafety.models import AnalyzeTextOptions

In [ ]:
for text in texts:
    print(f"Contnet: {text}")
    try:
        request = AnalyzeTextOptions(text=text)
        response = client.analyze_text(request)
    except HttpResponseError as e:
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    hate = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.HATE
    )
    self_harm = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.SELF_HARM
    )
    sexual = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.SEXUAL
    )
    violence = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.VIOLENCE
    )

    print("-" * 50)
    print(
        f"Hate: {hate.severity}, SelfHarm: {self_harm.severity}, Sexual: {sexual.severity}, Violence: {violence.severity}"
    )
    print("-" * 50)


In [125]:
from azure.ai.contentsafety.models import ImageCategory
from azure.ai.contentsafety.models import AnalyzeImageOptions, ImageData

In [ ]:
for url in images_urls:
    print(f"Contnet: {url}")
    request = AnalyzeImageOptions(image=ImageData(content=requests.get(url).content))

    try:
        response = client.analyze_image(request)
    except HttpResponseError as e:
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
        print(e)
        continue

    hate = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.HATE
    )
    self_harm = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.SELF_HARM
    )
    sexual = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.SEXUAL
    )
    violence = next(
        item
        for item in response.categories_analysis
        if item.category == TextCategory.VIOLENCE
    )

    print("-" * 50)
    print(
        f"Hate: {hate.severity}, SelfHarm: {self_harm.severity}, Sexual: {sexual.severity}, Violence: {violence.severity}"
    )
    print("-" * 50)
